###Before accessing the blob using mount point, make sure to give storage account contributor and blob contributor role to the registered app

In [0]:
for i in dbutils.fs.ls("dbfs:/mnt/dataengproj/bronze/SalesLT/"):
  print(i.name)

Address/
Customer/
CustomerAddress/
Product/
ProductCategory/
ProductDescription/
ProductModel/
ProductModelProductDescription/
SalesOrderDetail/
SalesOrderHeader/


In [0]:
table_names = []

for i in dbutils.fs.ls("mnt/dataengproj/bronze/SalesLT/"):
  table_names.append(i.name.split('/')[0])

####Changing ModifiedDate column to Date format instead of timestamp in every table using loop

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

for i in table_names:
  path = 'dbfs:/mnt/dataengproj/bronze/SalesLT/' + i + '/' + i + '.parquet'
  df = spark.read.format('parquet').load(path)
  columns = df.columns

  for col in columns:
    if "Date" in col or "date" in col:
      df = df.withColumn(col,date_format(from_utc_timestamp(df[col].cast(TimestampType()),"UTC"),"yyyy-MM-dd"))

  output_path = 'dbfs:/mnt/dataengproj/silver/SalesLT/' + i + '/'
  df.write.format("delta").mode("overwrite").save(output_path)

####All columns with date has been transformed into Date format instead of Timestamp and all tables were moved to Silver layer in delta table format

In [0]:
display(df)